# Inspect source light curves and skymaps

### Setup main notebook options

In [1]:
region_name = "ALL_10-800_LOOSE"  #"TDS" #"CAINGSGII_10-800"  #"TDS" #"WD" #"MDIS_10-800" # "TDS_ELAISN1"
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+"_cat.fits"
show_skymaps = True
show_sed = False
load_src_from_file = False
lc_block_srcs = 5 #Setup light curve plotting parameters

#srcs_ids = [2996 ,11821 ,55034 ,61491 ,64360 ,73354 ,76243 ,92867 ,104078 ,106745 ,124860 ,162490 ,182622 ,199884 ,226975 ,267514 ,588567 ,599887 ,789511 ,814134 ,] # WD and candidates, including GAIA info
srcs_ids = []
srcs_ids = [8215 ,32448 ,54184 ,98746 ,151796 ,166179 ,172775 ,193067 ,199832 ,259271 ,265150 ,297278 ,305192 ,388172 ,432606 ,472623 ,535864 ,541266 ,625693 ,1521738 ,1551422 ,2136829 ,] # WD ALL_10-800-LOOSE
srcs_ids = [4993 ,13580 ,14504 ,30475 ,50388 ,52994 ,79477 ,120265 ,138783 ,139480 ,162697 ,204419 ,221349 ,228958 ,236839 ,253214 ,
            253670 ,256433 ,261213 ,331466 ,340464 ,381336 ,390864 ,489430 ,498065 ,525310 ,532554 ,533185 ,540235 ,583838 ,639720 ,
            645215 ,645730 ,659787 ,666224 ,673175 ,680119 ,691408 ,714140 ,764827 ,1037284 ,1069077 ,1070938 ,1805505 ,1839206
            ,2527679 ,3074917 ,] # WD ALL_10-800-LOOSE
srcs_ids = [227 ,243 ,285 ,309 ,325 ,332 ,473 ,513 ,855 ,908 ,919 ,955 ,1033 ,1070 ,1073 ,1077 ,1091 ,1124 ,1293 ,1325 ,1372 ,1404 ,1405 ,1550 ,1553 ,1578 ,1655 ,1810 ,1897 ,2002 ,2003 ,2106 ,2158 ,2166 ,2210 ,2340 ,2365 ,2380 ,2394 ,2448 ,2470 ,2517 ,2527 ,2533 ,2586 ,2614 ,2618 ,2651 ,2653 ,2883 ,2924 ,3091 ,3296 ,3301 ,3319 ,3423 ,3637 ,3791 ,3842 ,4100 ,4111 ,4173 ,4196 ,4239 ,4284 ,4325 ,4350 ,4467 ,4501 ,4515 ,4530 ,4555 ,4562 ,4586 ,4621 ,4626 ,4635 ,4637 ,4650 ,4697 ,4705 ,4780 ,4920 ,4938 ,4943 ,4972 ,4987 ,5098 ,5221 ,5244 ,5476 ,5699 ,5780 ,5783 ,5868 ,5994 ,6029 ,6151 ,6162 ,6172 ,6241 ,6372 ,6384 ,6427 ,6438 ,6466 ,6488 ,6494 ,6536 ,6764 ,6830 ,6931 ,6936 ,6996 ,6998 ,7000 ,7028 ,7085 ,7310 ,7376 ,7388 ,7437 ,7493 ,7528 ,7565 ,7705 ,7708 ,7760 ,7792 ,7827 ,7828 ,7914 ,7926 ,7946 ,7988 ,8069 ,8190 ,8240 ,8322 ,8333 ,8355 ,8366 ,8408 ,8431 ,8467 ,8499 ,8545 ,8603 ,8775 ,8880 ,8917 ,8945 ,9094 ,9160 ,9232 ,9273 ,9326 ,9334 ,9455 ,9547 ,9585 ,9600 ,9615 ,9628 ,9633 ,9644 ,9645 ,9662 ,9679 ,9697 ,9700 ,9793 ,9810 ,9825 ,9846 ,9931 ,9955 ,10024 ,10042 ,10065 ,10068 ,10085 ,10116 ,10118 ,10127 ,10134 ,10235 ,10265 ,10328 ,10504 ,10624 ,10801 ,10824 ,10884 ,10974 ,11030 ,11058 ,11063 ,11073 ,11094 ,11200 ,11250 ,11299 ,11322 ,11336 ,11431 ,11445 ,11478 ,11480 ,11494 ,11570 ,11580 ,11592 ,11652 ,11731 ,11862 ,11920 ,11930 ,11994 ,11995 ,11998 ,12019 ,12075 ,12141 ,12303 ,12317 ,12370 ,12385 ,12441 ,12447 ,12452 ,12533 ,12721 ,12834 ,12838 ,12891 ,12899 ,12912 ,12922 ,12987 ,13085 ,13128 ,13155 ,13345 ,13447 ,13455 ,13479 ,13563 ,13580 ,13691 ,13704 ,13706 ,13707 ,13784 ,13794 ,13806 ,13826 ,13830 ,13868 ,13934 ,13948 ,14012 ,14219 ,14246 ,14355 ,14407 ,14477 ,15084 ,15106 ,15143 ,15234 ,15293 ,15357 ,15378 ,15450 ,15468 ,15623 ,15667 ,15940 ,15971 ,16170 ,16174 ,16390 ,16502 ,16543 ,16564 ,16712 ,16743 ,16783 ,16871 ,17019 ,17500 ,17516 ,17574 ,17615 ,17708 ,17854 ,17946 ,17978 ,18383 ,18396 ,18709 ,19363 ,19647 ,19756 ,19905 ,19971 ,20103 ,20267 ,20271 ,20345 ,20371 ,20618 ,20831 ,20951 ,21115 ,21478 ,21504 ,21549 ,21553 ,21568 ,21591 ,21622 ,21689 ,21720 ,21735 ,21973 ,21974 ,22078 ,22080 ,22284 ,22309 ,22321 ,22339 ,22428 ,22534 ,22562 ,22586 ,22755 ,22766 ,22801 ,22919 ,23174 ,23218 ,23362 ,23415 ,23450 ,23563 ,23565 ,23693 ,23708 ,23757 ,24028 ,24078 ,24174 ,24264 ,24291 ,24342 ,24409 ,24415 ,24459 ,24470 ,24492 ,24705 ,24733 ,24743 ,24792 ,24954 ,25087 ,25119 ,25178 ,25218 ,25240 ,25424 ,25474 ,25546 ,25621 ,25631 ,25649 ,25756 ,25820 ,25884 ,25994 ,26003 ,26072 ,26141 ,26181 ,26185 ,26222 ,26264 ,26318 ,26332 ,26502 ,26683 ,26973 ,27098 ,27234 ,27255 ,27278 ,27694 ,27725 ,27906 ,28156 ,28184 ,28250 ,28282 ,28293 ,28299 ,28381 ,28545 ,28765 ,29275 ,29431 ,29483 ,29488 ,29558 ,29646 ,29647 ,29741 ,29917 ,29953 ,30040 ,30079 ,30200 ,30218 ,30288 ,30303 ,30316 ,30320 ,30352 ,30434 ,30440 ,30451 ,30477 ,30565 ,30588 ,30601 ,30628 ,30725 ,30742 ,30794 ,30873 ,30983 ,31014 ,31230 ,31234 ,31280 ,31355 ,31358 ,31571 ,31625 ,31666 ,31755 ,31844 ,31895 ,32062 ,32121 ,32207 ,32240 ,32339 ,32792 ,32852 ,32865 ,33047 ,33138 ,33145 ,33283 ,33699 ,33729 ,33805 ,33919 ,33928 ,34290 ,34348 ,34736 ,34957 ,34992 ,35107 ,35281 ,35343 ,35399 ,35579 ,35622 ,35689 ,35977 ,36173 ,36451 ,36903 ,36928 ,36999 ,37397 ,37538 ,37614 ,37988 ,38394 ,38464 ,38523 ,38681 ,38925 ,39087 ,39209 ,39331 ,39370 ,39770 ,40017 ,40376 ,40490 ,40505 ,40510 ,40621 ,41498 ,41677 ,41993 ,42089 ,42167 ,42236 ,42397 ,42399 ,42599 ,42637 ,42694 ,42982 ,43157 ,43308 ,43491 ,43558 ,43727 ,44203 ,44328 ,44344 ,44698 ,44726 ,44740 ,44743 ,46050 ,46064 ,46084 ,46086 ,46187 ,46259 ,46339 ,46367 ,46374 ,46430 ,46439 ,46478 ,46494 ,46549 ,46553 ,46557 ,46601 ,46691 ,46779 ,46853 ,46874 ,46900 ,46916 ,46939 ,46956 ,47424 ,47435 ,47453 ,47535 ,47674 ,47910 ,47921 ,48106 ,48185 ,48249 ,48441 ,48531 ,48541 ,48609 ,48610 ,48633 ,48813 ,48908 ,48928 ,49093 ,49203 ,49301 ,49360 ,49513 ,49528 ,49560 ,49738 ,49869 ,49950 ,49965 ,50349 ,50360 ,50795 ,51096 ,51144 ,51182 ,51253 ,51324 ,51336 ,51400 ,51472 ,51563 ,51564 ,51791 ,51794 ,51884 ,51889 ,51897 ,51923 ,51994 ,52166 ,52176 ,52247 ,52263 ,52267 ,52317 ,52325 ,52408 ,52433 ,52458 ,52510 ,52692 ,52733 ,52779 ,52784 ,52860 ,52902 ,52926 ,52983 ,52997 ,53020 ,53080 ,53127 ,53128 ,53173 ,53188 ,53191 ,53204 ,53240 ,53256 ,53279 ,53347 ,53614 ,53856 ,53941 ,54025 ,54263 ,54310 ,54471 ,54571 ,54702 ,54704 ,54860 ,54958 ,55166 ,55247 ,55276 ,55355 ,55387 ,55534 ,55599 ,55640 ,55642 ,55691 ,55818 ,55850 ,55966 ,55984 ,56136 ,56173 ,56238 ,56308 ,56335 ,56339 ,56384 ,56385 ,56420 ,56451 ,56475 ,56487 ,56506 ,56546 ,56605 ,56724 ,56820 ,56992 ,57032 ,57209 ,57214 ,57272 ,57299 ,57305 ,57311 ,57433 ,57698 ,57760 ,57827 ,57858 ,57981 ,57997 ,58017 ,58019 ,58177 ,58186 ,58238 ,58323 ,58936 ,59090 ,59108 ,59229 ,59234 ,59353 ,59385 ,59392 ,59446 ,59476 ,59493 ,59516 ,59588 ,59648 ,59709 ,59776 ,59800 ,59802 ,59891 ,59898 ,59925 ,59957 ,60012 ,60073 ,60104 ,60163 ,60164 ,60177 ,60192 ,60206 ,60219 ,60221 ,60327 ,60373 ,60479 ,60636 ,60710 ,60728 ,61029 ,61033 ,61048 ,61103 ,61106 ,61160 ,61172 ,61198 ,61319 ,61418 ,61508 ,61666 ,61669 ,61726 ,61802 ,61837 ,61884 ,61992 ,62103 ,62104 ,62181 ,62190 ,62228 ,62239 ,62483 ,62514 ,62634 ,62766 ,62771 ,62808 ,62894 ,63280 ,63317 ,63388 ,63393 ,63400 ,63452 ,63645 ,63769 ,63882 ,63987 ,63988 ,64120 ,64135 ,64176 ,64308 ,64323 ,64523 ,64652 ,64681 ,64754 ,64770 ,64816 ,64826 ,65239 ,65271 ,65304 ,65422 ,65433 ,65608 ,65681 ,65806 ,65983 ,66085 ,66338 ,66342 ,66636 ,67073 ,67138 ,67152 ,67242 ,67283 ,67317 ,67369 ,67397 ,67406 ,67473 ,67477 ,67491 ,67514 ,67564 ,67592 ,67621 ,67637 ,67763 ,67792 ,67854 ,67910 ,68182 ,68255 ,68291 ,68334 ,68628 ,68750 ,68833 ,69060 ,69225 ,69320 ,69585 ,69612 ,69658 ,69724 ,69762 ,69815 ,69836 ,69893 ,69920 ,69958 ,69972 ,70091 ,70369 ,70435 ,70560 ,70666 ,70900 ,70914 ,71106 ,71131 ,71180 ,71380 ,71420 ,71468 ,71586 ,71683 ,71721 ,71771 ,71822 ,71843 ,71861 ,71886 ,71898 ,72184 ,72227 ,72275 ,72283 ,72319 ,72361 ,72453 ,72548 ,72662 ,72797 ,72895 ,72927 ,73061 ,73488 ,73497 ,73747 ,73769 ,73860 ,73929 ,73973 ,74000 ,74061 ,74080 ,74152 ,74180 ,74224 ,74633 ,74659 ,74701 ,74707 ,75045 ,75183 ,75452 ,75660 ,75681 ,76068 ,76277 ,76296 ,76357 ,76630 ,76704 ,76852 ,76928 ,77088 ,77550 ,77707 ,77736 ,77799 ,77982 ,78046 ,78116 ,78196 ,78322 ,78358 ,78774 ,78814 ,78913 ,79100 ,79102 ,79205 ,79222 ,79234 ,79456 ,79624 ,79636 ,79653 ,79765 ,79814 ,79892 ,79948 ,80073 ,80132 ,80265 ,80618 ,80622 ,80638 ,80655 ,80867 ,80903 ,80964 ,81072 ,81095 ,81260 ,81411 ,81483 ,81486 ,81488 ,81646 ,81977 ,82037 ,82629 ,83130 ,83200 ,83329 ,83362 ,84201 ,84403 ,84527 ,84538 ,84948 ,85251 ,85502 ,85662 ,85728 ,85772 ,85790 ,85798 ,85846 ,85899 ,86121 ,86163 ,86205 ,86260 ,86375 ,86674 ,86697 ,87171 ,87205 ,89840 ,89967 ,91175 ,91416 ,91531 ,91816 ,92258 ,92639 ,92862 ,92874 ,93123 ,93131 ,93143 ,93223 ,93233 ,93251 ,93318 ,93519 ,93546 ,93913 ,94183 ,94229 ,94291 ,94466 ,94537 ,94587 ,94599 ,94668 ,95022 ,95054 ,95118 ,95153 ,95156 ,95172 ,95238 ,95722 ,96641 ,96876 ,96913 ,97396 ,98858 ,99647 ,100541 ,100686 ,100974 ,102088 ,102505 ,102714 ,102903 ,103225 ,103428 ,103498 ,103637 ,103817 ,103858 ,104376 ,104834 ,104988 ,105189 ,105359 ,105506 ,105769 ,105776 ,105925 ,105982 ,106233 ,106785 ,107753 ,107924 ,107967 ,108230 ,108335 ,108668 ,108810 ,108832 ,109156 ,109430 ,109608 ,110070 ,110859 ,111161 ,111315 ,111345 ,111379 ,111471 ,111521 ,111766 ,111859 ,111905 ,111991 ,111998 ,114148 ,114420 ,115844 ,118407 ,118542 ,119780 ,120154 ,120168 ,120346 ,121016 ,121028 ,121110 ,121136 ,121269 ,121767 ,121789 ,121844 ,121910 ,121939 ,121998 ,122006 ,122031 ,122040 ,122098 ,122169 ,122245 ,122250 ,122302 ,122332 ,122414 ,122430 ,122525 ,122762 ,123292 ,124135 ,124276 ,124566 ,124621 ,124821 ,125042 ,125235 ,125264 ,125419 ,125424 ,125481 ,125823 ,126134 ,126156 ,126462 ,126716 ,126729 ,127025 ,127026 ,127419 ,128524 ,129372 ,130118 ,130697 ,131049 ,134014 ,134074 ,134146 ,134270 ,134898 ,136410 ,136515 ,136518 ,136773 ,137159 ,137229 ,137254 ,137698 ,138378 ,139424 ,142691 ,142787 ,143322 ,143332 ,143740 ,143844 ,143950 ,144136 ,144617 ,144618 ,144766 ,144771 ,144787 ,144850 ,145008 ,145081 ,145100 ,145364 ,145382 ,145623 ,145692 ,145809 ,146132 ,146173 ,146195 ,146479 ,146538 ,146548 ,146773 ,146993 ,147067 ,147262 ,147408 ,147622 ,148028 ,148057 ,148132 ,148228 ,148299 ,148342 ,148422 ,148445 ,148458 ,148660 ,148699 ,148798 ,148887 ,149060 ,149100 ,149224 ,149506 ,149589 ,149603 ,149785 ,149972 ,150216 ,150316 ,150348 ,150429 ,150446 ,150505 ,150722 ,150742 ,150787 ,150920 ,150936 ,150968 ,150981 ,151053 ,151141 ,151221 ,151227 ,151235 ,151295 ,151783 ,152052 ,152056 ,152173 ,152369 ,152390 ,152603 ,152769 ,153128 ,153141 ,153468 ,153619 ,154137 ,154416 ,155087 ,155218 ,155273 ,155357 ,155391 ,155476 ,155555 ,155769 ,155882 ,156015 ,156118 ,156417 ,156698 ,157357 ,157736 ,157775 ,158043 ,158349 ,158442 ,158530 ,158637 ,158667 ,158792 ,158863 ,159024 ,159035 ,159037 ,159045 ,159261 ,159307 ,159311 ,159416 ,159417 ,159421 ,159539 ,159606 ,159620 ,159648 ,159742 ,159746 ,159762 ,159775 ,159777 ,159833 ,159865 ,159871 ,159889 ,159937 ,159985 ,160004 ,160007 ,160010 ,160095 ,160119 ,160132 ,160170 ,160318 ,160645 ,160746 ,160934 ,160980 ,161055 ,161108 ,161175 ,161475 ,161618 ,161930 ,161941 ,161970 ,162078 ,162184 ,162193 ,162697 ,163516 ,163552 ,163868 ,164271 ,164379 ,164889 ,164913 ,165343 ,165487 ,165515 ,165869 ,165988 ,166201 ,166409 ,166528 ,166848 ,166852 ,166978 ,167017 ,167607 ,169138 ,169419 ,169490 ,169943 ,170125 ,170159 ,170181 ,170554 ,171498 ,171508 ,171899 ,171917 ,171929 ,171932 ,172449 ,172524 ,173168 ,173170 ,173498 ,173916 ,173936 ,174348 ,174433 ,174536 ,174610 ,174716 ,174819 ,175478 ,175633 ,175740 ,175912 ,176554 ,176631 ,176635 ,176888 ,176901 ,177033 ,177131 ,177173 ,177193 ,177194 ,177382 ,177508 ,177653 ,177714 ,177980 ,178161 ,178272 ,178302 ,178360 ,178839 ,178858 ,178892 ,179012 ,181643 ,182325 ,182707 ,183006 ,183726 ,183744 ,184110 ,184528 ,185709 ,185867 ,186055 ,186317 ,186489 ,186711 ,186812 ,187656 ,189800 ,191155 ,192119 ,192306 ,192500 ,192508 ,192521 ,192576 ,193151 ,193337 ,193360 ,193442 ,193534 ,193683 ,194233 ,194477 ,194576 ,194818 ,194838 ,195109 ,195286 ,195593 ,195595 ,195946 ,195958 ,196024 ,196232 ,196253 ,196265 ,196661 ,196723 ,196726 ,196909 ,196971 ,197005 ,197038 ,197070 ,197186 ,197395 ,197570 ,197855 ,198049 ,198214 ,198218 ,198246 ,198283 ,198315 ,198347 ,198370 ,198579 ,198596 ,198637 ,199111 ,199440 ,199520 ,199636 ,199653 ,199795 ,200084 ,200332 ,200391 ,200487 ,200498 ,200727 ,200986 ,201340 ,201361 ,201429 ,201668 ,201677 ,201848 ,202092 ,202110 ,202162 ,202270 ,202380 ,202399 ,202440 ,202482 ,202540 ,202610 ,202641 ,202663 ,202685 ,202744 ,202847 ,203081 ,203146 ,203195 ,203287 ,203399 ,203440 ,203446 ,203583 ,203628 ,203729 ,203745 ,203754 ,203835 ,203925 ,203926 ,204005 ,204020 ,204046 ,204115 ,204176 ,204296 ,204448 ,206892 ,207792 ,207991 ,208092 ,208304 ,209654 ,210220 ,210357 ,212265 ,213748 ,214037 ,214324 ,214330 ,214943 ,215227 ,215487 ,215672 ,216564 ,216856 ,216950 ,217350 ,217754 ,217805 ,217932 ,217988 ,218438 ,219416 ,219426 ,219486 ,219560 ,219628 ,219820 ,219942 ,220253 ,220278 ,220564 ,221712 ,221784 ,221823 ,221863 ,221919 ,222134 ,222149 ,222249 ,222369 ,222571 ,222710 ,222780 ,223292 ,223389 ,223407 ,223460 ,223598 ,223678 ,224077 ,224217 ,224296 ,224301 ,224468 ,224502 ,224833 ,224980 ,224990 ,225405 ,225467 ,225909 ,226365 ,226372 ,226440 ,226563 ,226730 ,226732 ,226935 ,227167 ,227412 ,227525 ,227671 ,227709 ,227753 ,227798 ,227904 ,228024 ,228112 ,228118 ,228209 ,228328 ,228380 ,228521 ,228530 ,228718 ,228867 ,228957 ,228958 ,228965 ,229229 ,229396 ,229424 ,229701 ,229768 ,229818 ,229902 ,229958 ,230001 ,230112 ,230225 ,230534 ,230564 ,230637 ,230760 ,230877 ,231009 ,231056 ,231058 ,231076 ,231286 ,231524 ,231773 ,231909 ,232056 ,232109 ,232310 ,232342 ,232517 ,232518 ,233061 ,233151 ,233342 ,233919 ,234568 ,235097 ,235201 ,236294 ,236438 ,239915 ,240160 ,240178 ,240277 ,240686 ,240860 ,241754 ,242181 ,246689 ,247056 ,248384 ,251050 ,253449 ,253530 ,253616 ,259010 ,259191 ,259249 ,259253 ,259394 ,259472 ,259476 ,259618 ,259677 ,259697 ,259770 ,259775 ,259844 ,259910 ,259944 ,259972 ,259995 ,260036 ,260184 ,260338 ,260789 ,260941 ,261004 ,261051 ,261185 ,261443 ,261507 ,261715 ,261790 ,262086 ,262134 ,262774 ,263281 ,263372 ,263385 ,263540 ,264128 ,264303 ,264919 ,265002 ,265029 ,265151 ,265213 ,265510 ,265515 ,265645 ,265806 ,266095 ,266127 ,266195 ,266338 ,266372 ,266468 ,266599 ,266752 ,266787 ,266982 ,267287 ,267440 ,267462 ,267970 ,268117 ,268288 ,268325 ,268501 ,268579 ,268732 ,268885 ,268981 ,269141 ,269235 ,269246 ,269552 ,269765 ,270137 ,270179 ,270581 ,270727 ,271047 ,271489 ,271532 ,271616 ,272004 ,272096 ,272097 ,272327 ,272978 ,273246 ,273406 ,273508 ,273516 ,273581 ,273643 ,273977 ,274027 ,274147 ,274294 ,274314 ,274513 ,274750 ,274765 ,274908 ,275479 ,275553 ,275839 ,275985 ,276455 ,276538 ,276672 ,277122 ,277164 ,277349 ,277448 ,277709 ,277815 ,277938 ,278095 ,278209 ,278242 ,278388 ,278436 ,278443 ,278537 ,278629 ,278886 ,278895 ,279068 ,279073 ,279154 ,279223 ,279355 ,279797 ,279847 ,280000 ,280049 ,280213 ,280282 ,280385 ,280547 ,280676 ,281091 ,281213 ,281666 ,281743 ,281865 ,282177 ,282374 ,282461 ,282463 ,282707 ,282755 ,282757 ,283166 ,283372 ,283864 ,284066 ,284637 ,284730 ,284765 ,284971 ,285072 ,285169 ,285476 ,285762 ,285887 ,286627 ,286632 ,286676 ,286685 ,286809 ,287235 ,287780 ,287832 ,288381 ,288412 ,289529 ,289767 ,289851 ,290183 ,290667 ,290809 ,292016 ,292450 ,293210 ,293334 ,293497 ,294093 ,294385 ,294607 ,294634 ,294862 ,297341 ,297355 ,297360 ,297414 ,298020 ,298262 ,298271 ,298449 ,301171 ,302805 ,303654 ,303898 ,304094 ,306080 ,306686 ,307453 ,307626 ,308187 ,308871 ,309052 ,309363 ,309400 ,309728 ,311651 ,313319 ,313578 ,314202 ,314757 ,315010 ,315561 ,315837 ,318262 ,318486 ,320521 ,320621 ,320746 ,320750 ,321262 ,321325 ,321530 ,323061 ,323812 ,323954 ,324026 ,324174 ,324465 ,324545 ,324639 ,324733 ,324891 ,324903 ,324958 ,325135 ,325268 ,325373 ,325403 ,325538 ,325624 ,325727 ,326009 ,326011 ,326125 ,326167 ,327444 ,327537 ,327599 ,327606 ,327907 ,328280 ,328471 ,328945 ,328994 ,329016 ,329033 ,329035 ,329064 ,329066 ,329067 ,329068 ,329189 ,329241 ,329251 ,329308 ,329400 ,329496 ,329581 ,329707 ,329754 ,329833 ,329848 ,329886 ,329933 ,330102 ,330108 ,330170 ,330239 ,330302 ,330333 ,330383 ,330396 ,330522 ,330698 ,330715 ,330823 ,331107 ,331162 ,331245 ,331260 ,331326 ,331598 ,331828 ,331974 ,332036 ,332128 ,332368 ,332417 ,332564 ,332569 ,332637 ,332880 ,332889 ,332954 ,332967 ,333304 ,333488 ,333491 ,333621 ,333658 ,333816 ,334526 ,334568 ,334632 ,334822 ,334829 ,334847 ,334945 ,335328 ,335384 ,335471 ,335537 ,335673 ,335873 ,335911 ,335952 ,336070 ,336085 ,336170 ,336429 ,336628 ,336754 ,336759 ,337015 ,337286 ,337508 ,338013 ,338125 ,338266 ,338334 ,338367 ,338409 ,338841 ,338912 ,338931 ,339030 ,339109 ,339325 ,339489 ,339514 ,340122 ,340272 ,340301 ,340457 ,340464 ,340859 ,340980 ,341309 ,341391 ,341441 ,341597 ,341624 ,341645 ,341711 ,341735 ,342009 ,342042 ,342101 ,342305 ,342569 ,342678 ,343086 ,343400 ,343647 ,344223 ,344301 ,344576 ,344954 ,344958 ,345097 ,345128 ,345408 ,346276 ,346454 ,346471 ,346629 ,346789 ,346841 ,346921 ,347049 ,347646 ,347904 ,348404 ,348511 ,348939 ,348988 ,349626 ,351766 ,353032 ,353218 ,353261 ,353405 ,353521 ,353889 ,353979 ,354476 ,354660 ,355077 ,355119 ,355234 ,355544 ,355748 ,356549 ,356634 ,356883 ,357258 ,358420 ,385783 ,388748 ,389222 ,389704 ,389771 ,389997 ,389999 ,390016 ,390044 ,390119 ,390230 ,390248 ,390346 ,390478 ,390864 ,390870 ,390893 ,390956 ,391353 ,391551 ,391748 ,391791 ,391823 ,391922 ,391980 ,392179 ,392224 ,392247 ,392288 ,392307 ,392480 ,392600 ,392670 ,392681 ,392822 ,392845 ,392906 ,393046 ,393194 ,393513 ,393605 ,393685 ,393723 ,393792 ,393821 ,393840 ,393841 ,393876 ,393924 ,394000 ,394030 ,394052 ,394138 ,394221 ,394268 ,394273 ,394347 ,394397 ,394490 ,394590 ,395009 ,396417 ,396694 ,397729 ,397976 ,398447 ,399452 ,399801 ,400039 ,400079 ,400188 ,400272 ,400393 ,400401 ,400452 ,400552 ,400737 ,400953 ,400965 ,401069 ,401211 ,401425 ,401450 ,401456 ,401483 ,401656 ,401669 ,401720 ,401826 ,401877 ,401902 ,402035 ,402149 ,402402 ,403219 ,403226 ,403745 ,403842 ,403877 ,403917 ,403994 ,404024 ,404148 ,404337 ,404838 ,405025 ,405459 ,405473 ,405635 ,407542 ,407553 ,407586 ,407678 ,407734 ,408036 ,408177 ,408236 ,408594 ,409049 ,409053 ,409214 ,409424 ,409492 ,409570 ,409790 ,410556 ,410557 ,410582 ,410663 ,410731 ,410781 ,411027 ,411123 ,411728 ,412681 ,413120 ,413302 ,413674 ,414051 ,414553 ,414963 ,415207 ,415417 ,416230 ,416342 ,417460 ,417777 ,418147 ,418164 ,419147 ,419472 ,420120 ,422482 ,422833 ,422843 ,422962 ,423006 ,423534 ,424261 ,424639 ,424959 ,425083 ,425383 ,425480 ,425579 ,425813 ,426065 ,426144 ,426153 ,426246 ,426248 ,426537 ,426726 ,426781 ,426946 ,427351 ,427534 ,427655 ,427764 ,427863 ,427890 ,427902 ,428081 ,428096 ,428295 ,430096 ,432431 ,432628 ,432912 ,433417 ,434299 ,435051 ,435191 ,435221 ,435307 ,435666 ,435790 ,436016 ,436561 ,436961 ,437014 ,437369 ,437625 ,437908 ,437962 ,438568 ,439007 ,439116 ,439200 ,439566 ,440257 ,440276 ,440443 ,440469 ,440472 ,441118 ,441163 ,441683 ,441971 ,442065 ,442438 ,442506 ,442625 ,442908 ,443132 ,443432 ,443525 ,443956 ,444169 ,444190 ,444343 ,444402 ,444521 ,445585 ,445612 ,446697 ,447101 ,447155 ,447311 ,447474 ,447551 ,447659 ,448345 ,448378 ,448903 ,449136 ,449556 ,449748 ,449802 ,449953 ,450210 ,450734 ,450982 ,451263 ,451914 ,452170 ,452331 ,452429 ,452532 ,453561 ,453872 ,454081 ,454308 ,455070 ,455182 ,458131 ,459394 ,459726 ,460044 ,461848 ,463517 ,467042 ,467166 ,467374 ,467575 ,467966 ,468498 ,469563 ,469964 ,470053 ,470308 ,470320 ,471680 ,471716 ,472064 ,472185 ,472459 ,473647 ,473775 ,474419 ,475321 ,475431 ,475949 ,476442 ,477090 ,477556 ,477723 ,480316 ,492090 ,494389 ,494514 ,494624 ,494880 ,495937 ,496423 ,496682 ,496889 ,497025 ,497369 ,498483 ,499170 ,499388 ,500034 ,500821 ,501592 ,501865 ,501957 ,502117 ,502377 ,503148 ,503279 ,503439 ,504045 ,504777 ,504852 ,506312 ,507623 ,508225 ,508404 ,509905 ,510247 ,510385 ,510745 ,511280 ,511809 ,511846 ,512450 ,512689 ,512710 ,512748 ,512752 ,512911 ,513558 ,513880 ,514315 ,514944 ,514984 ,515021 ,515025 ,515659 ,515827 ,516119 ,516392 ,516548 ,516728 ,516766 ,516834 ,517200 ,517726 ,517834 ,518040 ,518112 ,518235 ,518664 ,519097 ,519381 ,519428 ,519700 ,519765 ,520160 ,520226 ,520344 ,520899 ,531702 ,531815 ,531855 ,531984 ,532106 ,532116 ,533685 ,534291 ,534322 ,534366 ,534648 ,534735 ,535074 ,535079 ,536161 ,536702 ,536741 ,536781 ,537051 ,537841 ,537909 ,537953 ,537980 ,538075 ,538574 ,539008 ,539638 ,539996 ,541222 ,541386 ,541915 ,542049 ,542339 ,542663 ,543422 ,543490 ,544069 ,544473 ,544632 ,545065 ,545589 ,546118 ,546424 ,546690 ,546727 ,546776 ,546815 ,546817 ,546826 ,546837 ,546839 ,546908 ,546962 ,546964 ,546966 ,546970 ,547028 ,547032 ,547101 ,547112 ,547141 ,547159 ,547187 ,547193 ,547229 ,547258 ,547266 ,547298 ,547316 ,547338 ,547595 ,547718 ,547843 ,548052 ,548092 ,548390 ,548437 ,548456 ,548716 ,548992 ,549989 ,550223 ,550270 ,550941 ,551018 ,551412 ,552195 ,552535 ,552567 ,552676 ,552892 ,553170 ,553213 ,553663 ,553930 ,554069 ,554107 ,554253 ,554380 ,554386 ,554707 ,554859 ,555016 ,555076 ,555524 ,555949 ,556065 ,556154 ,556179 ,556234 ,557215 ,557603 ,557855 ,558261 ,558310 ,558491 ,558710 ,559020 ,559389 ,559797 ,560050 ,560188 ,560465 ,560470 ,560744 ,561460 ,561487 ,561944 ,561997 ,562243 ,563619 ,563640 ,563826 ,565270 ,565715 ,566430 ,566780 ,567506 ,567997 ,568429 ,568573 ,568915 ,569044 ,569097 ,569139 ,570841 ,571442 ,571619 ,572254 ,572664 ,572684 ,574606 ,575361 ,575796 ,575966 ,576190 ,576231 ,576271 ,576939 ,577088 ,577137 ,577307 ,577500 ,578275 ,578342 ,578651 ,578846 ,579038 ,579270 ,579493 ,579862 ,580124 ,580556 ,581316 ,581492 ,581632 ,581798 ,581877 ,581967 ,582237 ,582268 ,582343 ,582506 ,582618 ,582737 ,582746 ,583295 ,585162 ,585936 ,586355 ,591846 ,593012 ,593966 ,594027 ,594100 ,595310 ,596137 ,596925 ,596930 ,597356 ,597491 ,600420 ,600506 ,600964 ,601312 ,601512 ,602449 ,603443 ,604209 ,604646 ,604726 ,605178 ,607068 ,608241 ,608701 ,608740 ,609064 ,609110 ,609596 ,609674 ,609850 ,610129 ,610831 ,612171 ,612255 ,612678 ,612699 ,613238 ,613572 ,613755 ,613909 ,614345 ,614890 ,616032 ,617607 ,617782 ,619240 ,619718 ,619955 ,620562 ,621789 ,624102 ,624347 ,624363 ,626283 ,627021 ,628454 ,628637 ,629090 ,629513 ,630025 ,630286 ,630392 ,630781 ,630981 ,631340 ,631501 ,631704 ,631888 ,632098 ,632183 ,632459 ,632984 ,633429 ,633912 ,636068 ,636192 ,636892 ,637290 ,637340 ,637588 ,637643 ,638863 ,638887 ,639106 ,639822 ,640030 ,640193 ,642007 ,642090 ,642153 ,642677 ,642682 ,642703 ,642894 ,643213 ,643333 ,643482 ,643676 ,643762 ,643893 ,644545 ,644551 ,644724 ,644788 ,644828 ,645430 ,645467 ,645468 ,645564 ,645695 ,645838 ,646035 ,646157 ,646252 ,646301 ,646461 ,646501 ,647003 ,647226 ,647269 ,647512 ,647834 ,647963 ,648109 ,648228 ,648342 ,648383 ,648684 ,648791 ,649000 ,649020 ,649075 ,649143 ,649369 ,649449 ,649727 ,650403 ,650632 ,650659 ,650847 ,650902 ,651146 ,657024 ,666974 ,667376 ,667438 ,667492 ,667627 ,667669 ,667768 ,667839 ,667846 ,667873 ,667889 ,667911 ,668020 ,668066 ,668173 ,668179 ,668245 ,668672 ,668906 ,668991 ,669159 ,669213 ,669632 ,669975 ,670746 ,670956 ,670997 ,671569 ,672405 ,672566 ,672794 ,672867 ,672903 ,673032 ,673056 ,673082 ,673098 ,673100 ,673168 ,673210 ,673224 ,673318 ,673402 ,673409 ,673446 ,673660 ,673702 ,673737 ,673750 ,673907 ,673939 ,673943 ,673989 ,674323 ,674337 ,674548 ,674805 ,675028 ,675058 ,675410 ,675457 ,675526 ,675897 ,675914 ,676085 ,676145 ,676219 ,676364 ,676380 ,676534 ,676566 ,676596 ,676601 ,676655 ,676677 ,676858 ,676931 ,676959 ,677038 ,677380 ,677383 ,677395 ,678145 ,678380 ,678663 ,678664 ,678837 ,678913 ,679035 ,679075 ,679139 ,679210 ,679385 ,679428 ,679551 ,679639 ,679726 ,679780 ,680134 ,680934 ,680988 ,681421 ,681429 ,681464 ,681474 ,681627 ,681688 ,681742 ,681743 ,682000 ,682002 ,682174 ,682195 ,682234 ,682457 ,682576 ,682647 ,682826 ,682969 ,683069 ,683128 ,683297 ,683496 ,683604 ,683706 ,683723 ,683980 ,684135 ,684185 ,684348 ,684396 ,685226 ,685549 ,685563 ,685575 ,685599 ,685708 ,685714 ,685740 ,685799 ,685895 ,685896 ,686008 ,686203 ,686307 ,686311 ,686343 ,686348 ,686434 ,686439 ,686459 ,686494 ,686547 ,686578 ,686718 ,687117 ,687133 ,687135 ,687175 ,687233 ,687286 ,687292 ,687302 ,687391 ,687446 ,687634 ,687670 ,687845 ,688194 ,688208 ,688247 ,688331 ,688502 ,688732 ,689007 ,689058 ,689123 ,689126 ,689355 ,689390 ,689872 ,689950 ,690214 ,690415 ,690961 ,691014 ,691302 ,691547 ,691816 ,692005 ,692431 ,692464 ,692784 ,692891 ,693050 ,693185 ,693269 ,693406 ,693562 ,693666 ,693765 ,694118 ,694145 ,694212 ,694330 ,694401 ,694639 ,694672 ,694896 ,695425 ,695429 ,696067 ,696120 ,696407 ,696445 ,697172 ,697203 ,697279 ,697326 ,697572 ,697705 ,698080 ,698504 ,698743 ,698838 ,699104 ,699375 ,699616 ,699810 ,699950 ,699974 ,700005 ,700151 ,700255 ,700302 ,700393 ,700404 ,700618 ,700633 ,700665 ,700864 ,700983 ,701083 ,701093 ,701171 ,701279 ,701341 ,701455 ,701696 ,702276 ,702345 ,702478 ,702511 ,702674 ,702898 ,702899 ,702996 ,703149 ,703336 ,703344 ,703426 ,703536 ,703544 ,703677 ,703764 ,703844 ,703881 ,704628 ,704643 ,704711 ,704752 ,704810 ,705143 ,705199 ,705394 ,705532 ,705790 ,705845 ,706253 ,706256 ,706399 ,706588 ,706678 ,706847 ,706953 ,706981 ,707069 ,707084 ,707297 ,707328 ,707333 ,707414 ,707550 ,708112 ,708139 ,708309 ,708485 ,708517 ,708634 ,708692 ,708717 ,708842 ,709034 ,709166 ,709331 ,709335 ,709714 ,710333 ,710428 ,710450 ,711020 ,711743 ,711875 ,711995 ,712013 ,712153 ,712244 ,712584 ,713627 ,713632 ,714153 ,714283 ,715174 ,715483 ,716020 ,717960 ,718709 ,719409 ,720104 ,720608 ,720659 ,720966 ,722760 ,722827 ,722832 ,722984 ,723546 ,723979 ,724696 ,725278 ,725737 ,726248 ,726405 ,726591 ,726784 ,727922 ,728134 ,728321 ,728946 ,728978 ,729127 ,729882 ,729910 ,729991 ,730138 ,730274 ,730277 ,730339 ,730448 ,730640 ,731154 ,731177 ,731678 ,731776 ,731987 ,741307 ,741382 ,741658 ,741965 ,742514 ,742630 ,742872 ,742926 ,743695 ,743803 ,743912 ,753720 ,754068 ,754377 ,755765 ,756091 ,756119 ,756282 ,756682 ,756892 ,757245 ,759977 ,761439 ,761885 ,762351 ,772197 ,772383 ,776979 ,778278 ,780285 ,783465 ,783895 ,785807 ,786462 ,797079 ,801899 ,803149 ,803547 ,806790 ,844950 ,845062 ,845631 ,846726 ,877244 ,878657 ,881656 ,882083 ,883945 ,897269 ,898746 ,901717 ,905047 ,907750 ,916586 ,923517 ,945957 ,946176 ,1024458 ,1031813 ,1036492 ,1061556 ,1073686 ,1075688 ,1084980 ,1086554 ,1106654 ,1111091 ,1112528 ,1115293 ,1116253 ,1116446 ,1168576 ,1169421 ,1171118 ,1175164 ,1179316 ,1181606 ,1183312 ,1185970 ,1187478 ,1191082 ,1193693 ,1194687 ,1197830 ,1203128 ,1203787 ,1206692 ,1208090 ,1209891 ,1210196 ,1211440 ,1213331 ,1213574 ,1214165 ,1214552 ,1215576 ,1215825 ,1215962 ,1215966 ,1217085 ,1219015 ,1220555 ,1222361 ,1224898 ,1226216 ,1228231 ,1228516 ,1229666 ,1230673 ,1230765 ,1231540 ,1231828 ,1233030 ,1233488 ,1233524 ,1234157 ,1235949 ,1236466 ,1236593 ,1238108 ,1240478 ,1240877 ,1243398 ,1245576 ,1249571 ,1259156 ,1262794 ,1265495 ,1265560 ,1267194 ,1267227 ,1269081 ,1270926 ,1271961 ,1272368 ,1272786 ,1274449 ,1278193 ,1278222 ,1284883 ,1285162 ,1285917 ,1286107 ,1287296 ,1288663 ,1288926 ,1290004 ,1290925 ,1292334 ,1293133 ,1295169 ,1296463 ,1300090 ,1301157 ,1309846 ,1312358 ,1318790 ,1327493 ,1328421 ,1329834 ,1331418 ,1332086 ,1334675 ,1334712 ,1334724 ,1337037 ,1337616 ,1339564 ,1342181 ,1343572 ,1349546 ,1353446 ,1354208 ,1354611 ,1361013 ,1361948 ,1364400 ,1366015 ,1367233 ,1382689 ,1408498 ,1412034 ,1416459 ,1417054 ,1427467 ,1439734 ,1468570 ,1469385 ,1469771 ,1469826 ,1470569 ,1472471 ,1472650 ,1472932 ,1473677 ,1473792 ,1473972 ,1474604 ,1475179 ,1476332 ,1478642 ,1480348 ,1482789 ,1483143 ,1483385 ,1488012 ,1490421 ,1491988 ,1539429 ,1540818 ,1542523 ,1542934 ,1543776 ,1544526 ,1544990 ,1545822 ,1549420 ,1553083 ,1554067 ,1555740 ,1556043 ,1556253 ,1559835 ,1559852 ,1560155 ,1561498 ,1562535 ,1563035 ,1563449 ,1565299 ,1567005 ,1569121 ,1569407 ,1569919 ,1572773 ,1572866 ,1573595 ,1577350 ,1578603 ,1578882 ,1579732 ,1579840 ,1580275 ,1581737 ,1582188 ,1584522 ,1585411 ,1586566 ,1588073 ,1588804 ,1590791 ,1592262 ,1609724 ,1612545 ,1624264 ,1624572 ,1625785 ,1626212 ,1626533 ,1630957 ,1637503 ,1638586 ,1638627 ,1639464 ,1642201 ,1643075 ,1643727 ,1644232 ,1645018 ,1646374 ,1647025 ,1647184 ,1647992 ,1648811 ,1650401 ,1651290 ,1651850 ,1652148 ,1653580 ,1654885 ,1655369 ,1656150 ,1656576 ,1656803 ,1657320 ,1657562 ,1657842 ,1658604 ,1658865 ,1659574 ,1660915 ,1664651 ,1664778 ,1667135 ,1668138 ,1668331 ,1671184 ,1672139 ,1672849 ,1673720 ,1687758 ,1697519 ,1704092 ,1711016 ,1714291 ,1717464 ,1720618 ,1724416 ,1837186 ,1840800 ,1864702 ,1864995 ,1871162 ,1875404 ,1876398 ,1876832 ,1877112 ,1877494 ,1879257 ,1879310 ,1881508 ,1881920 ,1882638 ,1884477 ,1885672 ,1893810 ,1894884 ,1897630 ,1898282 ,1898995 ,1901515 ,1902191 ,1902362 ,1902426 ,1903809 ,1907189 ,1907966 ,1910140 ,1911183 ,1914181 ,1915514 ,1917225 ,1928245 ,1938386 ,1948486 ,1951450 ,1953256 ,1954213 ,1956650 ,1962851 ,1971267 ,1973087 ,1983552 ,1994663 ,2027968 ,2035200 ,2035816 ,2037328 ,2041853 ,2042836 ,2066342 ,2069250 ,2070479 ,2080958 ,2085716 ,2088315 ,2090376 ,2091293 ,2091568 ,2091625 ,2094427 ,2094903 ,2095358 ,2097196 ,2097323 ,2101075 ,2101909 ,2103750 ,2105391 ,2106440 ,2109733 ,2113119 ,2115923 ,2116541 ,2117520 ,2120301 ,2125483 ,2132131 ,2137338 ,2140546 ,2144507 ,2149595 ,2149741 ,2150084 ,2156586 ,2157677 ,2164936 ,2165146 ,2165648 ,2165847 ,2182895 ,2189610 ,2194097 ,2211541 ,2219162 ,2222633 ,2222901 ,2223332 ,2267221 ,2332808 ,2343507 ,2349217 ,2350716 ,2351982 ,2352295 ,2353172 ,2362393 ,2363615 ,2373002 ,2374000 ,2376462 ,2376905 ,2385674 ,2386614 ,2399170 ,2401354 ,2402287 ,2404483 ,2409607 ,2411632 ,2434737 ,2475916 ,2477456 ,2489797 ,2491604 ,2497909 ,2501776 ,2502578 ,2524383 ,2528318 ,2530928 ,2542050 ,2544519 ,2591747 ,2613847 ,2614723 ,2614791 ,2614874 ,2616671 ,2617045 ,2617889 ,2618719 ,2618973 ,2619822 ,2621096 ,2621412 ,2623032 ,2623979 ,2624862 ,2661031 ,2664799 ,2669017 ,2669105 ,2670740 ,2677708 ,2687309 ,2689183 ,2698884 ,2711986 ,2775976 ,2778623 ,2821063 ,2833417 ,2841187 ,2843738 ,2851986 ,2862717 ,2883275 ,2900682 ,2906199 ,2906216 ,2906335 ,2908279 ,2911876 ,2913843 ,2918568 ,2919730 ,2920400 ,2921203 ,2921296 ,2924131 ,2926292 ,2926770 ,2927320 ,2927583 ,2927790 ,2929796 ,2931094 ,2932019 ,2936383 ,2936522 ,2937357 ,2937697 ,2940508 ,2942355 ,2943274 ,2943639 ,2944475 ,2945045 ,2945871 ,2946300 ,2946524 ,2947426 ,2955406 ,2956321 ,3035825 ,3076627 ,3083503 ,3093505 ,3115678 ,3117732 ,3125041 ,3142423 ,3146735 ,3146895 ,3149295 ,3152250 ,3153083 ,3156323 ,3174206 ,3177474 ,3179747 ,3183854 ,3191432 ,3194992 ,3199278 ,3203759 ,3212350 ,3224680 ,3271696 ,3272327 ,]

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord
from astropy.table import Table
import numpy as np

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux, sel_sources, dd_filter2id

from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord

#Load region
rg = Region()
rg.load_from_fits(region_fname)

print("Nr sources:",len(rg.tt_sources),"selected:",rg.tt_sources["sel"].sum())
    
#Subselect sources based on choice
if len(srcs_ids)>0:
    rg.tt_sources.add_index("rg_src_id")
    idx_srcs = rg.tt_sources.loc_indices["rg_src_id", srcs_ids]
    tt_srcs = Table(rg.tt_sources[idx_srcs])
else:
    tt_srcs = rg.tt_sources

nr_srcs = len(tt_srcs)
tt_srcs.add_index("rg_src_id")
#display(tt_srcs)

Nr sources: 10413 selected: 10413


### Show all info for one source

In [3]:
#Define displayed tables and columns
disp_tt_sources = ['rg_src_id','nr_det','ra','dec','pos_err','pos_xv','pos_cpval','pos_rchiq','coadd_src_id','coadd_dist','obs_filter_id','flux','flux_err','flux_nxv','flux_cpval','flux_rchiq','coadd_ffactor',
                   'coadd_fdiff_s2n','nr_fd_srcs','hr','hr_err'] #'fd_src_id','rg_fd_id,,'pos_var','flux_var','sel', ,'simbad_match_id','gaiadr3_match_id'
disp_tt_simbad = ['ogrp','otypes','main_id','otype','ra','dec','ra_prec','dec_prec','coo_err_maja','coo_err_mina','coo_err_angle','coo_qual','coo_wavelength','coo_bibcode','distance_distance','distance_q','distance_unit',
                  'distance_merr','distance_perr','distance_method','distance_bibcode','match_distance','rvz_type','rvz_radvel','rvz_error','rvz_qual','rvz_wavelength','rvz_bibcode','z_value','sp_type','sp_qual',
                  'sp_bibcode','simbad_match_id'] #,'rg_src_id',,'script_number_id'
disp_tt_gaiadr3 = ['ogrp','Gmag_abs','BP-RP','ra','dec','Source','e_RA_ICRS','e_DE_ICRS','Plx','e_Plx','PM','pmRA','e_pmRA','pmDE','e_pmDE','RUWE','FG','e_FG','Gmag','FBP','e_FBP','BPmag','FRP','e_FRP','RPmag','RV','e_RV',
                   'Vbroad','GRVSmag','QSO','Gal','NSS','XPcont','XPsamp','RVS','EpochPh','EpochRV','MCMCGSP','MCMCMSC','And','Teff','logg','__Fe_H_','Dist','A0','HIP','PS1','SDSS13','SKYM2','TYC2','URAT1','AllWISE','APASS9',
                   'GSC23','RAVE5','_2MASS','RAVE6','RAJ2000','DEJ2000','PQSO','PGal','PSS','RPlx','match_distance','gaiadr3_match_id'] #,'_q',,'rg_src_id'
dd_tab_col = {"tt_sources":disp_tt_sources, "tt_simbad":disp_tt_simbad, "tt_gaiadr3":disp_tt_gaiadr3}

def display_src_info(rg_src_id=0):

    #Get source
    tc_src = rg.get_src_from_id(rg_src_id, load_from_file = load_src_from_file, add_sed=show_sed,add_gphoton=False, add_spectrum=show_sed)

    #Prepare figure

    fig, axs = plt.subplots(1,2,figsize=(20, 6),squeeze=True, num="Source ID " + str(rg_src_id), clear=True, gridspec_kw={'width_ratios': [2, 1]})
    axs = axs.flatten()
    
    #Plot light curve und SED
    vvis.plot_light_curve(tc_src, ax = axs[0]) #
    if show_sed:
        vvis.plot_sed(tc_src,ax = axs[1])
    plt.show()

    for table_name, col_names in dd_tab_col.items():
        if table_name in tc_src._table_names:
            print("\n",table_name,":")
            display(tc_src.__dict__[table_name][col_names])

    #If lomb scargle present and signal is at more than 3 sigma plot it too
    if "tt_lombscargle" in tc_src._table_names:
        if tc_src.tt_lombscargle["ls_peak_pval"] < 0.002699796063:
            display(tc_src.tt_lombscargle)
            fig_ls, axs_ls = plt.subplots(1,2,figsize=(12, 6),squeeze=True, num="LS " + str(rg_src_id), clear=True)
            axs_ls = axs_ls.flatten()
            vvis.plot_lombscargle(tc_src.tt_source_lc, ax=axs_ls[0], ax_lc = axs[0])
            vvis.plot_lombscargle(tc_src.tt_source_lc, ax=axs_ls[1],obs_filter="FUV")
            plt.show()
    
    #tc_src.tt_sed.pprint_all()

widgets.interactive(display_src_info, rg_src_id=tt_srcs["rg_src_id"])

interactive(children=(Dropdown(description='rg_src_id', options=(227, 243, 285, 309, 325, 332, 473, 513, 855, …

### Draw light curves

In [4]:
nr_blocks = int(nr_srcs/lc_block_srcs)+1 
def plot_lc_range(block_nr):
    fig_lc = plt.figure("Light curves",figsize=(10, 9), clear=True)
    lc_min = lc_block_srcs * block_nr
    lc_max = lc_min + lc_block_srcs if lc_min + lc_block_srcs<=nr_srcs-1 else nr_srcs
    vvis.plot_light_curves(rg,fig=fig_lc, rg_src_ids = tt_srcs["rg_src_id"][lc_min:lc_max], ylim = [1.2,550])
    plt.show()
    #display(tt_srcs[lc_min:lc_max]) #[show_lc_pars]
widgets.interactive( plot_lc_range, block_nr=(0,nr_blocks-1))

interactive(children=(IntSlider(value=370, description='block_nr', max=740), Output()), _dom_classes=('widget-…

### Plot skymaps for one source

In [5]:
def plot_sky_src(rg_src_id=0, fd_idx=0, vis_idx = -1):
    
    #Check if src_id exists
    if rg_src_id not in tt_srcs["rg_src_id"]:
        print("Source ID does not exist") 
        return
    
    #Get source and check id field index out of range
    tc_src = rg.get_src_from_id(rg_src_id)
    if fd_idx > len(tc_src.tt_fields)-1:
        print("Field index out of range") 
        return
    
    #Get field to plot and check if visit index out of range
    rg_fd_id = tc_src.tt_fields[fd_idx]["rg_fd_id"]
    fd_src = rg.get_field(rg_fd_id=rg_fd_id, load_method="FITS", add_field=True)
    if vis_idx > len(fd_src.tt_visits)-1:
        print("Visit index out of range") 
        return

    fig_src = plt.figure("Sky map source",figsize=(8, 7), clear=True)

    #Get sky region of source
    sky_center = SkyCoord(tc_src.tt_sources[0]["ra"]*uu.deg, tc_src.tt_sources[0]["dec"]*uu.deg, frame="icrs")
    sky_region_src = CircleSkyRegion(center=sky_center, radius=0.3*uu.arcmin)
     
    #Plot
    plt.cla()
    ax_src, wcs_src = vvis.plot_field_sky_map(fd_src, img_idx=vis_idx, sky_region = sky_region_src, fig=fig_src) #
    
    #If displaying visits, onle show detections in that visit
    sel_det_rg = np.ones(len(rg.tt_detections), dtype=bool)
    sel_det_src = np.ones(len(tc_src.tt_detections), dtype=bool)
    
    # Preselt field detections, can be removed in the future
    #fd_src.tt_detections = fd_src.tt_detections[fd_src.tt_detections["s2n"]>0]
    
    sel_det_fd = np.ones(len(fd_src.tt_detections), dtype=bool)
    if vis_idx > -1:
        vis_id = fd_src.tt_visits["vis_id"][vis_idx]
        sel_det_rg = (rg.tt_detections["vis_id"]==vis_id) * (rg.tt_detections["obs_filter_id"]==dd_filter2id[fd_src.obs_filter])
        sel_det_src = (tc_src.tt_detections["vis_id"]==vis_id) * (tc_src.tt_detections["obs_filter_id"]==dd_filter2id[fd_src.obs_filter])

        fd_det_coords = SkyCoord(fd_src.tt_detections["ra"][sel_det_fd], fd_src.tt_detections["dec"][sel_det_fd], frame="icrs")
        sel_det_fd = ( (fd_src.tt_detections["vis_id"] == vis_id)
                     * (fd_src.tt_detections["obs_filter_id"] == dd_filter2id[fd_src.obs_filter])
                     * sky_region_src.contains(fd_det_coords, wcs_src))
        
    #Show sources
    plt_ref_src_kwargs = {"marker": "x","markersize": 10,"color":"yellow"}
    _ = vvis.plot_sky_sources(rg.tt_sources,tt_det=rg.tt_detections[sel_det_rg], sky_region_wcs = (sky_region_src,wcs_src),only_selected=False) #
    _ = vvis.plot_sky_sources(rg.tt_coadd_sources,tt_det=None, sky_region_wcs = (sky_region_src,wcs_src),only_selected=False, src_id="coadd_src_id", src_kwargs = plt_ref_src_kwargs)
    if vis_idx > -1:
        plot_tds = {"marker":"x","color":"green"}
        _ = vvis.plot_sky_sources(fd_src.tt_detections[sel_det_fd],src_kwargs=plot_tds, sky_region_wcs = (sky_region_src,wcs_src),  src_id="det_id",only_selected=False)
    plt.show()

    # Display sources only for the shown field
    #fd_src_id = rg.tt_src_id_map[sel_fd_src]["fd_src_id"][0]
    if vis_idx > -1:
        print("\nAll detections for this visit in the field:")
        display(fd_src.tt_detections[sel_det_fd])
    
#Decide which sources to show
list_srcs = tt_srcs["rg_src_id"]
if len(srcs_ids)>0:
    list_srcs = srcs_ids

if show_skymaps:
    widget = widgets.interactive( plot_sky_src, rg_src_id=list_srcs,
                        fd_idx=widgets.IntSlider(min=0, max=10, value=0, step=1),
                        vis_idx = widgets.IntSlider(min=-1, max=100, value=-1, step=1)) #visprob_srcids = [1669,1801,2815,3776,4338,6137,] 
    
    controls = widgets.HBox(widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
    output = widget.children[-1]
    display(widgets.VBox([controls, output]))
    # straight NUV, dashed FUV

### Draw sky maps of entire fields

In [6]:
def plot_sky_field(rg_fd_id):
    if rg_fd_id not in rg.tt_fields["rg_fd_id"]:
        print("source ID does not exist") 
        return
    fig_skymap = plt.figure("Sky map",figsize=(9, 8), clear=True) #, constrained_layout=True
    fd = rg.get_field(rg_fd_id=rg_fd_id, load_method="FITS", add_field=True)
    ax, wcs = vvis.plot_field_sky_map(fd, img_idx=-1, sky_region = fd.get_sky_region(),fig=fig_skymap) # 
    ax, tt_src_shown = vvis.plot_sky_sources(rg.tt_sources,tt_det=rg.tt_detections, sky_region_wcs = (fd.get_sky_region(),wcs),only_selected=True) # 
    plt.show()
    #display(tt_src_shown) #[show_map_pars]
if show_skymaps: 
    widgets.interactive(plot_sky_field, rg_fd_id=rg.tt_fields["rg_fd_id"])